In [42]:
import networkx as nx
from pyvis.network import Network

import pandas as pd

In [47]:
classesDF  = pd.read_csv('../../data/elliptic_txs_classes.csv')

# Remove all nodes that have unknown class
classesDF = classesDF[classesDF['class'] != "unknown"]

# If class = 2, then map to 0, else map to 1
classesDF['class'] = classesDF['class'].map({'2': 0, '1': 1})

print(f'The dataset has {len(classesDF)} vectors.')

The dataset has 46564 vectors.


In [58]:
edgesDF  = pd.read_csv('../../data/elliptic_txs_edgelist.csv')

# From classesDF, remove 90% of the nodes of each class
reducedDF = classesDF.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.2))

print(f'The dataset that will be displayed has {len(reducedDF)} vectors.')

# Remove all edges that belong to a node that is not in classesDF
edgesDF = edgesDF[edgesDF['txId1'].isin(reducedDF['txId'])]
edgesDF = edgesDF[edgesDF['txId2'].isin(reducedDF['txId'])]

# Create a directed graph
G = nx.from_pandas_edgelist(edgesDF, 'txId1', 'txId2', create_using=nx.DiGraph)

# Add the class to the nodes
txid_to_class = classesDF.set_index('txId')['class'].to_dict()
for node in G.nodes():
    if node in txid_to_class:
        G.nodes[node]['class'] = txid_to_class[node]
    else:
        G.nodes[node]['class'] = None  # Or some default value if the txId is not found

The dataset that will be displayed has 9313 vectors.


In [60]:
# Print the class of each node
for node in G.nodes(data=True):
    print(f"Node {node[0]}: Class {node[1]['class']}")
    break

Node 232440111: Class 0


# Plot the graph

In [61]:
# Initialize PyVis network
nt = Network('720px', '1080px')

# Populate the nodes and edges data structures
nt.from_nx(G)

# Optionally, add more visualization options here
for node in G.nodes(data=True):
    title = f"Class: {node[1]['class']}"
    nt.get_node(node[0])['title'] = title
    nt.get_node(node[0])['group'] = node[1]['class']

# Save and display the network
nt.save_graph('graph_interface.html')

# Example

In [8]:
from pyvis.network import Network
import networkx as nx
nx_graph = nx.cycle_graph(10)
nx_graph.nodes[1]['title'] = 'Number 1'
nx_graph.nodes[1]['group'] = 1
nx_graph.nodes[3]['title'] = 'I belong to a different group!'
nx_graph.nodes[3]['group'] = 10
nx_graph.add_node(20, size=20, title='couple', group=2)
nx_graph.add_node(21, size=15, title='couple', group=2)
nx_graph.add_edge(20, 21, weight=5)
nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
nt = Network('720px', '1080px')
# populates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.save_graph('nx.html')